In [1]:
# Imports
import pandas as pd
import requests
from io import BytesIO, StringIO
from zipfile import ZipFile
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import numpy as np

In [2]:
# Watermark
print('Nathan Young\nJunior Data Analyst\nCenter for the Study of Free Enterprise')
%load_ext watermark
%watermark -a "Western Carolina University" -u -d -p pandas

Nathan Young
Junior Data Analyst
Center for the Study of Free Enterprise
Western Carolina University 
last updated: 2020-02-05 

pandas 1.0.0


In [3]:
# Load BEA CAINC5N_NC data
response = requests.get('https://apps.bea.gov/regional/zip/CAINC5N.zip')
zip_file = ZipFile(BytesIO(response.content))
files = zip_file.namelist()
with zip_file.open(files[34]) as csvfile:
    df = pd.read_csv(csvfile, encoding='ISO-8859-1', sep=",")

In [4]:
# Check for unused fields
df.tail(10)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
13225,"""37199""","Yancey, NC",5.0,CAINC5N,2000.0,...,Government and government enterprises,Thousands of dollars,31811,32905,...,42039,43069,42119,41528,40990,40595,42823,50157,51335,51005
13226,"""37199""","Yancey, NC",5.0,CAINC5N,2001.0,...,Federal civilian,Thousands of dollars,2992,3157,...,3041,3715,3545,3446,2180,2227,3371,3270,3197,3318
13227,"""37199""","Yancey, NC",5.0,CAINC5N,2002.0,...,Military,Thousands of dollars,652,881,...,1768,1687,1633,1480,1389,1287,1220,1261,1221,1308
13228,"""37199""","Yancey, NC",5.0,CAINC5N,2010.0,...,State and local,Thousands of dollars,28167,28867,...,37230,37667,36941,36602,37421,37081,38232,45626,46917,46379
13229,"""37199""","Yancey, NC",5.0,CAINC5N,2011.0,...,State government,Thousands of dollars,5028,5552,...,6036,5825,5599,5728,5874,5297,5224,5508,5512,5843
13230,"""37199""","Yancey, NC",5.0,CAINC5N,2012.0,...,Local government,Thousands of dollars,23139,23315,...,31194,31842,31342,30874,31547,31784,33008,40118,41405,40536
13231,Note: See the included footnote file.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13232,CAINC5N: Personal Income by Major Component an...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13233,"Last updated: November 14, 2019-- new statisti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13234,Source: U.S. Department of Commerce / Bureau o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Remove unused fields
df.drop(df.tail(4).index,inplace=True)

In [6]:
#Clean GeoFIPS
df['GeoFIPS'] = df['GeoFIPS'].replace({"":''})

In [7]:
# Set GeoFIPS as Index
df.set_index(df['GeoFIPS'], inplace = True)

In [8]:
# Drop GeoFIPS column
df.drop('GeoFIPS', axis = 1, inplace = True)

In [ ]:
#Connect to database and create cursor
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=TITANIUM-BOOK;'
                      'Database=DataDashboard;'
                      'Trusted_Connection=yes;',
                    autocommit=True)

c = con.cursor()

# Create Per Capita Personal Income

In [9]:
# Create Backups
df_pc_backup = pd.read_csv('./Updates/STG_BEA_Per_Capita_Personal_Income.txt', encoding = 'ISO-8859-1', sep='\t')
df_pc_backup.to_csv('./Backups/STG_BEA_Per_Capita_Personal_Income_BACKUP.txt')

In [10]:
# Create new dataframe for Per capita personal income
filter1 = df['Description'].str.contains("Per capita")
df_per_capita = df[filter1]

In [11]:
# Save as tab-delimited txt file for export to SSMS
df_per_capita.to_csv('../Updates/STG_BEA_Per_Capita_Personal_Income.txt', sep = '\t')

In [16]:
# Reset the index
df_per_capita == df.reset_index()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,"""37000""",North Carolina,5.0,CAINC5N,10.0,...,Personal income (thousands of dollars),Thousands of dollars,228879962,232267281,...,338315713,341627602,355052267,379925288,376023857,397995923,419891523,433195584,453769026,478861557
1,"""37000""",North Carolina,5.0,CAINC5N,20.0,...,Population (persons) 2/,Number of persons,8210122,8326201,...,9449566,9574293,9656754,9749123,9843599,9933944,10033079,10156679,10270800,10383620
2,"""37000""",North Carolina,5.0,CAINC5N,30.0,...,Per capita personal income (dollars),Dollars,27878,27896,...,35802,35682,36767,38970,38200,40064,41851,42651,44180,46117
3,"""37000""",North Carolina,5.0,CAINC5N,35.0,...,Earnings by place of work,Thousands of dollars,174113335,176206894,...,243168101,243762027,247464422,267072169,266760383,280636099,293911001,303689403,317893541,333559983
4,"""37000""",North Carolina,5.0,CAINC5N,36.0,...,Less: Contributions for government social ins...,Thousands of dollars,20100040,20369137,...,27492664,27812781,25926513,26956299,31307775,32936145,34551393,35605704,36993403,38219633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,"""37199""","Yancey, NC",5.0,CAINC5N,2001.0,...,Federal civilian,Thousands of dollars,2992,3157,...,3041,3715,3545,3446,2180,2227,3371,3270,3197,3318
13227,"""37199""","Yancey, NC",5.0,CAINC5N,2002.0,...,Military,Thousands of dollars,652,881,...,1768,1687,1633,1480,1389,1287,1220,1261,1221,1308
13228,"""37199""","Yancey, NC",5.0,CAINC5N,2010.0,...,State and local,Thousands of dollars,28167,28867,...,37230,37667,36941,36602,37421,37081,38232,45626,46917,46379
13229,"""37199""","Yancey, NC",5.0,CAINC5N,2011.0,...,State government,Thousands of dollars,5028,5552,...,6036,5825,5599,5728,5874,5297,5224,5508,5512,5843


In [13]:
# Fill NaN values for upload to database
column_list = df_per_capita.columns.values
for i in column_list:
    df_per_capita.loc[df_per_capita[i].isnull(),i]=0

In [14]:
# Drop old backup table
#c.execute('drop table STG_BEA_Per_Capita_Personal_Income_BACKUP')

In [15]:
# Create new backup
c.execute('''sp_rename 'dbo.STG_BEA_Per_Capita_Personal_Income','STG_BEA_Per_Capita_Personal_Income_BACKUP';''')

NameError: name 'c' is not defined

In [ ]:
# Create Per Capita table
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_BEA_Per_Capita_Personal_Income](
	[GeoFIPS] [varchar](12) NULL,
	[GeoName] [varchar](14) NULL,
	[Region] [real] NULL,
	[TableName] [varchar](7) NULL,
	[LineCode] [real] NULL,
	[IndustryClassification] [varchar](3) NULL,
	[Description] [varchar](38) NULL,
	[Unit] [varchar](20) NULL,
	[2001] [float] NULL,
	[2002] [float] NULL,
	[2003] [float] NULL,
	[2004] [float] NULL,
	[2005] [float] NULL,
	[2006] [float] NULL,
	[2007] [float] NULL,
	[2008] [float] NULL,
	[2009] [float] NULL,
	[2010] [float] NULL,
	[2011] [float] NULL,
	[2012] [float] NULL,
	[2013] [float] NULL,
	[2014] [float] NULL,
	[2015] [float] NULL,
	[2016] [float] NULL,
	[2017] [float] NULL,
	[2018] [float] NULL,
    [2019] [float] NULL,
    [2020] [float] NULL,
    [2021] [float] NULL,
    [2022] [float] NULL,
    [2023] [float] NULL,
    [2024] [float] NULL,
    [2025] [float] NULL
) ON [PRIMARY]''')

In [ ]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#df: pandas.dataframe; mTableName:table name in MS SQL
#warning: discard old table if exists
df.to_sql('STG_BEA_Per_Capita_Personal_Income', con=engine, if_exists='replace', index=False)

# Create Earnings by Place of Work

In [ ]:
# Create Backups
df_e_backup = pd.read_csv('./Updates/STG_BEA_Earnings_by_Place_of_Work.txt', encoding = 'ISO-8859-1', sep='\t')
df_e_backup.to_csv('./Backups/STG_BEA_Earnings_by_Place_of_Work_BACKUP.txt')

In [ ]:
# Create a new dataframe for Earnings by place of work
filter1 = df['Description'].str.contains('Earnings')
df_earnings = df[filter1]

In [ ]:
# Save as tab-delimited txt file for export to SSMS
df_earnings.to_csv('./Updates/STG_BEA_Earnings_by_Place_of_Work.txt', sep = '\t')

In [ ]:
# Reset the index
df_earnings == df.reset_index()

In [ ]:
# Fill NaN values for upload to database
column_list = df_earnings.columns.values
for i in column_list:
    df_earnings.loc[df_earnings[i].isnull(),i]=0

In [ ]:
# Drop old backup table
#c.execute('drop table STG_BEA_Earnings_by_Place_of_Work_BACKUP')

In [ ]:
# Create new backup
c.execute('''sp_rename 'dbo.STG_BEA_Earnings_by_Place_of_Work','STG_BEA_Earnings_by_Place_of_Work_BACKUP';''')

In [ ]:
# Create Earnings table
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_BEA_Earnings_by_Place_of_Work](
	[GeoFIPS] [varchar](12) NULL,
	[GeoName] [varchar](14) NULL,
	[Region] [real] NULL,
	[TableName] [varchar](7) NULL,
	[LineCode] [real] NULL,
	[IndustryClassification] [varchar](3) NULL,
	[Description] [varchar](38) NULL,
	[Unit] [varchar](20) NULL,
	[2001] [float] NULL,
	[2002] [float] NULL,
	[2003] [float] NULL,
	[2004] [float] NULL,
	[2005] [float] NULL,
	[2006] [float] NULL,
	[2007] [float] NULL,
	[2008] [float] NULL,
	[2009] [float] NULL,
	[2010] [float] NULL,
	[2011] [float] NULL,
	[2012] [float] NULL,
	[2013] [float] NULL,
	[2014] [float] NULL,
	[2015] [float] NULL,
	[2016] [float] NULL,
	[2017] [float] NULL,
	[2018] [float] NULL,
    [2019] [float] NULL,
    [2020] [float] NULL,
    [2021] [float] NULL,
    [2022] [float] NULL,
    [2023] [float] NULL,
    [2024] [float] NULL,
    [2025] [float] NULL
) ON [PRIMARY]''')

In [ ]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#df: pandas.dataframe; mTableName:table name in MS SQL
#warning: discard old table if exists
df.to_sql('STG_BEA_Earnings_by_Place_of_Work', con=engine, if_exists='replace', index=False)

# Create Population

In [ ]:
# Create Backups
df_pop_backup = pd.read_csv('./Updates/STG_BEA_Population.txt', encoding = 'ISO-8859-1', sep='\t')
df_pop_backup.to_csv('./Backups/STG_BEA_Population_BACKUP.txt')

In [ ]:
# Create a new dataframe for Population
filter1 = df['Description'].str.contains('Population')
df_population = df[filter1]

In [ ]:
# Clean Description column
df_population.loc[:,'Description'] = df_population['Description'].str.strip('2/')

In [ ]:
# Save as tab-delimited txt file for export to SSMS
df_population.to_csv('./Updates/STG_BEA_Population.txt', sep = '\t')

In [ ]:
# Reset the index
df_population == df.reset_index()

In [ ]:
# Fill NaN values for upload to database
column_list = df_population.columns.values
for i in column_list:
    df_population.loc[df_population[i].isnull(),i]=0

In [ ]:
# Drop old backup table
#c.execute('drop table STG_BEA_Population_BACKUP')

In [ ]:
# Create new backup
c.execute('''sp_rename 'dbo.STG_BEA_Population','STG_BEA_Population_BACKUP';''')

In [ ]:
# Create Population table
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_BEA_Population](
	[GeoFIPS] [varchar](12) NULL,
	[GeoName] [varchar](14) NULL,
	[Region] [real] NULL,
	[TableName] [varchar](7) NULL,
	[LineCode] [real] NULL,
	[IndustryClassification] [varchar](3) NULL,
	[Description] [varchar](38) NULL,
	[Unit] [varchar](20) NULL,
	[2001] [float] NULL,
	[2002] [float] NULL,
	[2003] [float] NULL,
	[2004] [float] NULL,
	[2005] [float] NULL,
	[2006] [float] NULL,
	[2007] [float] NULL,
	[2008] [float] NULL,
	[2009] [float] NULL,
	[2010] [float] NULL,
	[2011] [float] NULL,
	[2012] [float] NULL,
	[2013] [float] NULL,
	[2014] [float] NULL,
	[2015] [float] NULL,
	[2016] [float] NULL,
	[2017] [float] NULL,
	[2018] [float] NULL,
    [2019] [float] NULL,
    [2020] [float] NULL,
    [2021] [float] NULL,
    [2022] [float] NULL,
    [2023] [float] NULL,
    [2024] [float] NULL,
    [2025] [float] NULL
) ON [PRIMARY]''')

In [ ]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#df: pandas.dataframe; mTableName:table name in MS SQL
#warning: discard old table if exists
df.to_sql('STG_BEA_Population', con=engine, if_exists='replace', index=False)

# Create Personal Income

In [ ]:
# Create Backups
df_i_backup = pd.read_csv('./Updates/STG_BEA_Personal_Income.txt', encoding = 'ISO-8859-1', sep='\t')
df_i_backup.to_csv('./Backups/STG_BEA_Personal_Income_BACKUP.txt')

In [ ]:
# Create new dataframe for Personal Income
filter1 = df['Description'].str.contains('Personal income')
df_income = df[filter1]

In [ ]:
# Save as tab-delimited txt file for export to SSMS
df_income.to_csv('./Updates/STG_BEA_Personal_Income.txt', sep = '\t')

In [ ]:
# Reset the index
df_income == df.reset_index()

In [ ]:
# Fill NaN values for upload to database
column_list = df_income.columns.values
for i in column_list:
    df_income.loc[df_income[i].isnull(),i]=0

In [ ]:
# Drop old backup table
#c.execute('drop table STG_BEA_Personal_Income_BACKUP')

In [ ]:
# Create new backup
c.execute('''sp_rename 'dbo.STG_BEA_Personal_Income','STG_BEA_Personal_Income_BACKUP';''')

In [ ]:
# Create Personal Income Table
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_BEA_Personal_Income](
	[GeoFIPS] [varchar](12) NULL,
	[GeoName] [varchar](14) NULL,
	[Region] [real] NULL,
	[TableName] [varchar](7) NULL,
	[LineCode] [real] NULL,
	[IndustryClassification] [varchar](3) NULL,
	[Description] [varchar](38) NULL,
	[Unit] [varchar](20) NULL,
	[2001] [float] NULL,
	[2002] [float] NULL,
	[2003] [float] NULL,
	[2004] [float] NULL,
	[2005] [float] NULL,
	[2006] [float] NULL,
	[2007] [float] NULL,
	[2008] [float] NULL,
	[2009] [float] NULL,
	[2010] [float] NULL,
	[2011] [float] NULL,
	[2012] [float] NULL,
	[2013] [float] NULL,
	[2014] [float] NULL,
	[2015] [float] NULL,
	[2016] [float] NULL,
	[2017] [float] NULL,
	[2018] [float] NULL,
    [2019] [float] NULL,
    [2020] [float] NULL,
    [2021] [float] NULL,
    [2022] [float] NULL,
    [2023] [float] NULL,
    [2024] [float] NULL,
    [2025] [float] NULL
) ON [PRIMARY]''')

In [ ]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#df: pandas.dataframe; mTableName:table name in MS SQL
#warning: discard old table if exists
df.to_sql('STG_BEA_Personal_Income', con=engine, if_exists='replace', index=False)